In [47]:
from cobra.io import read_sbml_model
from pathlib import Path
import os
import cobra

cobra.Configuration().solver = "gurobi"

# change working directory to context_specific_models folder
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# create a dictionary with the filenames (keys) and models (values)
folder = Path().cwd()
files = [f for f in os.listdir(folder / 'cs_mods') if '.xml'in f]
mod = read_sbml_model(folder / 'cs_mods' / 'Human-GEM.xml')

In [50]:
import pandas as pd

# add back any lost exchange reactions
def add_all_EX_rxns(basis_model,model):
    EX_rxns = [r for r in basis_model.reactions if len(r.products)==0]
    EX_rxns_model = [r.id for r in model.reactions if len(r.products)==0]
    missing_rxns = [r.copy() for r in EX_rxns if r.id not in EX_rxns_model]
    model.add_reactions(missing_rxns)
    return model

# import excel file with info on exchange upper bounds 
# file contains 'id' col with IDs and an 'upper_bound' col with upper bounds to modify
bounds = pd.read_excel(folder / 'rxn_bounds' / 'R3D301_EX_rxns.xlsx')
bounds = bounds[bounds['upper_bound']>0] # select entries with non-zero upper bounds

# add ATP demand reactions, lost exchange reactions to all models
for r in [r for r in mod.reactions if len(r.products)==0]: r.bounds = (0,1000)     # close lower bound for all exchange reactions (metabolites can leave the system but not enter)
for i in range(len(bounds)): mod.reactions.get_by_id(bounds['id'][i]).bounds = (-bounds['upper_bound'][i],1000)     # set bounds Exchange reactions (lower bounds -1000, upper bounds based on file info)

In [51]:
from cobra.io import write_sbml_model
#mod_test = read_sbml_model(folder / 'cs_mods' / files[0] )
write_sbml_model(mod, "mod_test.xml")
mod_test = read_sbml_model(folder / 'cs_mods' / 'Human-GEM.xml')

In [3]:
# make RPE-PR model

# copy RPE, PR, and full Human1 models
mod = models['Human-GEM.xml'].copy()
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

def remove_compartment(model, compartment):
# remove compartment from model
    rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
    mets_n = [m for m in model.metabolites if compartment in m.compartment]
    model.remove_reactions(rxns_n)
    model.remove_metabolites(mets_n)
    return model

# remove nucleus from PR model
mod_PR = remove_compartment(mod_PR, 'n')

def add_id_suffix(model, suffix):
    for r in model.reactions:
        r.id = r.id + suffix
    for m in model.metabolites:
        m.id = m.id + suffix
        m.compartment = m.compartment + suffix
    return model
# add '_PR' to PR model IDs and '_RPE' to RPE model IDs
mod_PR = add_id_suffix(mod_PR, '_PR')
mod_RPE = add_id_suffix(mod_RPE, '_RPE')

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpz7c5ggz3.lp
Reading time = 0.05 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp66ahi42_.lp
Reading time = 0.03 seconds
: 5517 rows, 14194 columns, 54382 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpotvxy2c4.lp
Reading time = 0.02 seconds
: 5013 rows, 12490 columns, 46478 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [4]:
# add RPE-PR interface reactions to RPE model (rxn_RPE-PR = RPE <--> RPE-PR interface)
for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
    rxn = rxns.copy()
    rxn.id = rxn.id.replace('_RPE','_RPE-PR')
    for p in rxn.products:
        if 'e_RPE' in p.id:
            p.id = p.id.replace('e_RPE','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_RPE' in r.id:
            r.id = r.id.replace('e_RPE','e_RPE-PR')
            r.compartment = 'e_RPE-PR'
    mod_RPE.add_reactions([rxn])
    
# add RPE-PR interface reactions to PR model (rxn_PR-RPE = PR <--> RPE-PR interface)
for rxn in [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:
    rxn.id = rxn.id.replace('_PR','_PR-RPE')
    for p in rxn.products:
        if 'e_PR' in p.id:
            p.id = p.id.replace('e_PR','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_PR' in r.id:
            r.id = r.id.replace('e_PR','e_RPE-PR')
            r.compartment = 'e_RPE-PR'

In [5]:
# fuse RPE and PR model
from cobra import Model
mod_RPE_PR = Model('RPE-PR model')
mod_RPE_PR.add_reactions(mod_RPE.reactions)
mod_RPE_PR.add_reactions(mod_PR.reactions)

In [6]:
# list reaction IDs to be deleted (because their reactions are duplicated)
import pandas as pd
df = pd.DataFrame([[r.id, r.reaction] for r in mod_RPE_PR.reactions \
                   if r.compartments == {'e_RPE-PR'}], columns=['id','reaction'])
l  = list(df['id'][df['reaction'].duplicated()])

# delete duplicated reactions and change reaction IDs (suffix: _eRPE-PR) 
# of the reactions that only involve the e_RPE-PR compartment (interface)
for r in [r for r in mod_RPE_PR.reactions if r.compartments=={'e_RPE-PR'}]: 
    if r.id in l:
        mod_RPE_PR.remove_reactions(r.id)
    elif '_RPE-PR' in r.id:
        r.id = r.id.replace('_RPE-PR','_eRPE-PR')
    elif '_PR-RPE' in r.id:
        r.id = r.id.replace('_PR-RPE','_eRPE-PR')
        ,
def fix_compartment_dict(model):
    new_compartment_dict = model.compartments.copy()
    for k in list(new_compartment_dict.keys()):
        new_compartment_dict[k] = k 
    model.compartments = new_compartment_dict
    return model
fix_compartment_dict(mod_RPE_PR)

# close lower bounds of exchange reactions into RPE-PR interface space (nothing allowed in from outside)
rxns = [r for r in mod_RPE_PR.reactions if len(r.products)==0 if r.lower_bound<0 if 'eRPE-PR' in r.id]
for rxn in rxns: rxn.lower_bound = 0

C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


Name,RPE-PR model
Memory address,1b7dccbf820
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,0
Objective expression,0
Compartments,"c_RPE, x_RPE, m_RPE, e_RPE, l_RPE, g_RPE, r_RPE, n_RPE, i_RPE, e_RPE-PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [45]:
from cobra.io import write_sbml_model
#mod_test = read_sbml_model(folder / 'cs_mods' / files[0] )
write_sbml_model(models[k[4]], "mod_test.xml")

TypeError: in method 'Reaction_setReversible', argument 2 of type 'bool'

In [8]:
# open ATP demand reactions
# mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds=(0,1000)
# mod_RPE_PR.reactions.get_by_id('MAR03964_PR').bounds=(0,1000)
mod_RPE_PR.reactions.get_by_id('MAR03964_PR')

Reaction identifier,MAR03964_PR
Name,ATP phosphohydrolase
Memory address,0x1b7dc6757b0
Stoichiometry,MAM01371c_PR + MAM02040c_PR --> MAM01285c_PR + MAM02039c_PR + MAM02751c_PR ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
[r for r in mod_RPE_PR.reactions if len(r.products)==0][0]

Reaction identifier,MAR07108_RPE
Name,
Memory address,0x1b7d7433670
Stoichiometry,MAM01374e_RPE --> benzo[a]pyrene -->
GPR,
Lower bound,0
Upper bound,1000


In [25]:
mod_RPE_PR.reactions.get_by_id('MAR03964_PR')

Reaction identifier,MAR03964_PR
Name,ATP phosphohydrolase
Memory address,0x1b7dc6757b0
Stoichiometry,MAM01371c_PR + MAM02040c_PR --> MAM01285c_PR + MAM02039c_PR + MAM02751c_PR ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,0.0
Upper bound,1000.0


In [26]:
mod_RPE_PR.reactions.get_by_id('MAR09034_eRPE-PR')

Reaction identifier,MAR09034_eRPE-PR
Name,
Memory address,0x1b7e06e2f80
Stoichiometry,MAM01965e_RPE-PR --> glucose -->
GPR,
Lower bound,0
Upper bound,1000


In [22]:
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

Reaction identifier,MAR09034_RPE
Name,
Memory address,0x1b7d74600d0
Stoichiometry,MAM01965e_RPE <=> glucose <=>
GPR,
Lower bound,-40.0
Upper bound,1000


In [23]:
mod_RPE_PR.reactions.get_by_id('MAR09034_eRPE-PR')

Reaction identifier,MAR09034_eRPE-PR
Name,
Memory address,0x1b7e06e2f80
Stoichiometry,MAM01965e_RPE-PR --> glucose -->
GPR,
Lower bound,0
Upper bound,1000


In [29]:
# OBJECTIVE 
# set maximizing the ATP demand in PR as objective
mod_RPE_PR.objective = mod_RPE_PR.reactions.get_by_id('MAR03964_PR').flux_expression
mod_RPE_PR.objective.expression
solution = mod_RPE_PR.optimize()
print(solution)
mod_RPE_PR.summary()

<Solution 83.215 at 0x1b7e4bcdde0>


Metabolite,Reaction,Flux,C-Number,C-Flux
MAM01965e_RPE,MAR09034_RPE,40,6,83.05%
MAM02184e_RPE,MAR09039_RPE,0.5077,6,1.05%
MAM02360e_RPE,MAR09040_RPE,0.6538,6,1.36%
MAM03135e_RPE,MAR09046_RPE,1.179,5,2.04%
MAM01975e_RPE,MAR09063_RPE,2.651,5,4.59%
MAM01365e_RPE,MAR09066_RPE,0.2615,6,0.54%
MAM01986e_RPE,MAR09067_RPE,0.7031,2,0.49%
MAM01974e_RPE,MAR09071_RPE,1.338,5,2.32%
MAM02658e_RPE,MAR09087_RPE,0.9333,5,1.61%
MAM01587e_RPE,MAR09286_RPE,0.6564,6,1.36%


In [ ]:
# with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    try: solution = mod_RPE_PR.optimize()
#    except: print('model is infeasible')
mod_RPE_PR.summary()

In [ ]:
# FVA
#from cobra.flux_analysis import flux_variability_analysis
#flux_variability_analysis(mod_RPE_PR,loopless=True)

In [ ]:
# list all exchange and demand reactions matching Human1 and VMH IDs
def get_vmh_id(reaction):
    if 'vmhreaction' in reaction.annotation.keys(): vmh_id = reaction.annotation['vmhreaction']
    else: vmh_id = ''
    return vmh_id
df_EX_rxns = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products) == 0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns

In [ ]:
# ATP metabolites
ATP_mets = [[met.id,met.name] for met in mod_RPE_PR.metabolites if met.name=='ATP']
pd.DataFrame(ATP_mets,columns=['ID','name'])

# ATP reactions
ATP_rxns = [[r.id,get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
            for r in mod_RPE_PR.reactions if 'MAM01371' in r.reaction]

pd.DataFrame(ATP_rxns,columns=['id','vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

In [ ]:
# list all exchange and demand reactions
df_EX_rxns_human1 = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products)==0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns.to_clipboard(excel=True, sep=None)
df_EX_rxns

In [ ]:
# match VMH_IDs from bounds file to Human1 IDs to create 'R3D301_EX_rxns_RPE-PR.xlsx'

# import excel file with info on bounds
bounds = pd.read_excel(folder / 'rxn_bounds' / 'files PL' / 'R3D301_EX_rxns.xlsx')

# vmh_id lookup table + look up RPE rxn
df_vmh_id = pd.DataFrame([[get_vmh_id(r),r.id,r.id+'_RPE'] for r in mod.reactions if 'EX_' in get_vmh_id(r)],columns=['vmh_id','human1_id','id'])
merged_df = df_vmh_id.merge(bounds, how = 'right', on = ['vmh_id'])
merged_df.to_clipboard(excel=True, sep=None)
merged_df

In [ ]:
# biomass reaction
mod.reactions.get_by_id('MAR04413') 

pd.DataFrame([[r.id,r.name,r.reaction,r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
              for r in mod_RPE_PR.reactions if 'biomass' in r.name])

In [ ]:
mod.reactions.get_by_id('MAR13082') 